In [1]:
pip install tonic-validate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
import os
from langchain.document_loaders import DirectoryLoader
from dotenv import load_dotenv
from langchain.llms import OpenAI
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import pandas as pd

In [3]:
#text splitting
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(texts[1]) 

page_content='According to a study from the University of Cambridge, at least half of developers’ efforts are spent debugging and not actively programming, which costs the software industry an estimated $312 billion per year. But so far, only a handful of code-generating AI systems have been made freely available to the public — reflecting the commercial incentives of the organizations building them (see: Replit).\n\nStarCoder, which by contrast is licensed to allow for royalty-free use by anyone, including corporations, was trained on over 80 programming languages as well as text from GitHub repositories, including documentation and programming notebooks. StarCoder integrates with Microsoft’s Visual Studio Code code editor and, like OpenAI’s ChatGPT, can follow basic instructions (e.g., “create an app UI”) and answer questions about code.' metadata={'source': 'new_articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt'}


In [4]:
#load env variables(keys}
load_dotenv()

True

In [5]:
from openai import OpenAI
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
# embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
embedding_function = OpenAIEmbeddings(
api_key=OPENAI_API_KEY)

In [6]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=OpenAIEmbeddings())

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=vectordb.as_retriever())

In [8]:
#Load 10 questions and answers about the Paul Graham essays as a benchmark for how the RAG system should answer questions.
with open("question_and_answer_list.json", "r") as f:
    question_and_answer_list =json.load(f)

In [22]:
ex_q_and_a = question_and_answer_list[1]
ex_q_and_a["question"]

'What is ChatGPT, and how has it been used in various applications?'

In [23]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

query = ex_q_and_a["question"]
matching_docs =vectordb.similarity_search(query)
print(matching_docs)

[Document(page_content='What is the ChatGPT character limit?\n\nIt’s not documented anywhere that ChatGPT has a character limit. However, users have noted that there are some character limitations after around 500 words.\n\nDoes ChatGPT have an API?\n\nYes, it was released March 1, 2023.\n\nWhat are some sample everyday uses for ChatGPT?\n\nEveryday examples include programing, scripts, email replies, listicles, blog ideas, summarization, etc.\n\nWhat are some advanced uses for ChatGPT?\n\nAdvanced use examples include debugging code, programming languages, scientific concepts, complex problem solving, etc.\n\nHow good is ChatGPT at writing code?\n\nIt depends on the nature of the program. While ChatGPT can write workable Python code, it can’t necessarily program an entire app’s worth of code. That’s because ChatGPT lacks context awareness — in other words, the generated code isn’t always appropriate for the specific context in which it’s being used.\n\nCan you save a ChatGPT chat?', m

In [24]:
# response = chain(ex_q_and_a["question"])
response=chain.run(input_documents=matching_docs, question=query)
print(response)

ChatGPT is a general-purpose chatbot developed by OpenAI that uses artificial intelligence to generate text in response to user prompts. It is powered by GPT-4, a large language model that employs deep learning to produce human-like text. 

ChatGPT has been used in various applications including programming, scripting, email replies, listicles, blog ideas, summarization, and more. It can engage in conversations on a wide range of topics such as programming languages, scientific concepts, and complex problem solving. Users have found it useful for debugging code and generating workable Python code. However, it's important to note that ChatGPT lacks context awareness, so the generated code may not always be appropriate for specific contexts.

In addition to these everyday and advanced uses, ChatGPT has also been employed in other areas such as generating TV scripts and engaging in discussions on scientific concepts. Its versatility and AI-powered capabilities make it a valuable tool for 

In [25]:
ex_q_and_a["answer"]

'ChatGPT is a text-generating AI chatbot developed by OpenAI. It has been widely used for writing essays, code, and more based on short text prompts, enhancing productivity. Major brands have experimented with it for generating ad and marketing copy. OpenAI continually invests in ChatGPT, upgrading it to GPT-4, a more advanced language-writing model. The chatbot has been integrated into various applications, including search engines, customer service, and even an iPhone customization app called SuperChat.'

In [26]:
from tonic_validate.validate_scorer import ValidateScorer 
# metrics
from tonic_validate.metrics.answer_consistency_metric import AnswerConsistencyMetric
from tonic_validate.metrics.answer_similarity_metric import AnswerSimilarityMetric
from tonic_validate.metrics.augmentation_accuracy_metric import AugmentationAccuracyMetric
from tonic_validate.metrics.augmentation_precision_metric import AugmentationPrecisionMetric
from tonic_validate.metrics.retrieval_precision_metric import RetrievalPrecisionMetric
# llm utils
from tonic_validate.classes.llm_response import LLMResponse
from tonic_validate.classes.benchmark import BenchmarkItem

In [27]:
metrics = [
    AnswerSimilarityMetric(),
    RetrievalPrecisionMetric(),
    AugmentationAccuracyMetric(),
    AugmentationPrecisionMetric(),
    AnswerConsistencyMetric()
]
# can use an OpenAI chat completion model
# llm_evaluator = "gpt-3.5-turbo"
llm_evaluator = "gpt-4-1106-preview"
validate_scorer = ValidateScorer(
    metrics, llm_evaluator
)


In [28]:
# example BenchmarkItem
question = ex_q_and_a["question"]
reference_answer = ex_q_and_a["answer"]
benchmark_item = BenchmarkItem(
    question=question,
    answer=reference_answer
)

# example LLMResponse
llm_answer = response
context_list = [document.page_content for document in matching_docs]
llm_response = LLMResponse(
    llm_answer=llm_answer,
    llm_context_list=context_list,
    benchmark_item=benchmark_item
)

responses = [llm_response]

response_scores = validate_scorer.score_run(responses)

In [29]:
response_scores.run_data[0].scores

{'answer_similarity': 4.0,
 'retrieval_precision': 1.0,
 'augmentation_accuracy': 0.25,
 'augmentation_precision': 0.25,
 'answer_consistency': 1.0}

In [17]:
def make_scores_df(response_scores):
    scores_df = {
        "question": [],
        "reference_answer": [],
        "llm_answer": [],
        "retrieved_context": []
    }
    for score_name in response_scores.overall_scores:
        scores_df[score_name] = []
    for data in response_scores.run_data:
        scores_df["question"].append(data.reference_question)
        scores_df["reference_answer"].append(data.reference_answer)
        scores_df["llm_answer"].append(data.llm_answer)
        scores_df["retrieved_context"].append(data.llm_context)
        for score_name, score in data.scores.items():
            scores_df[score_name].append(score)
    return pd.DataFrame(scores_df)
            


In [18]:
scores_df = make_scores_df(response_scores)

In [21]:
scores_df.head()

,question,reference_answer,llm_answer,retrieved_context,answer_similarity,retrieval_precision,augmentation_accuracy,augmentation_precision,answer_consistency
0,What does Pando plan to use the $30 million ra...,Pando intends to use the funds for expanding i...,Pando plans to use the $30 million raised in i...,[Signaling that investments in the supply chai...,4.0,0.5,0.25,0.5,1.0
